# Collection of Functions within notebook to then port to specific .py files

Useful reference: http://mathesaurus.sourceforge.net/idl-numpy.html


## APOD3D Helping function

In [28]:
import numpy as np

def do_apod3d(nx, ny, nz, squared):

    if squared:
        apodx = np.sin(np.pi*(np.arange(0, nx, 1, dtype=float)+0.5)/nx) ** 2 
        apody = np.sin(np.pi*(np.arange(0, ny, 1, dtype=float)+0.5)/ny) ** 2 
        apodz = np.sin(np.pi*(np.arange(0, nz, 1, dtype=float)+0.5)/nz) ** 2

        # TODO: Figure out what is meant to happen with these vectors
        apodxy = np.multiply(apodx, apody)
        apodxy = np.concatenate((apodxy, [nx,ny,nz]))
        apodz2 = np.transpose(np.concatenate((apodz, [nz, nz, ny])))

        apodxyz = apodz2

    return apodxyz

do_apod3d(20,20,20, True)

array([6.15582970e-03, 5.44967379e-02, 1.46446609e-01, 2.73004750e-01,
       4.21782767e-01, 5.78217233e-01, 7.26995250e-01, 8.53553391e-01,
       9.45503262e-01, 9.93844170e-01, 9.93844170e-01, 9.45503262e-01,
       8.53553391e-01, 7.26995250e-01, 5.78217233e-01, 4.21782767e-01,
       2.73004750e-01, 1.46446609e-01, 5.44967379e-02, 6.15582970e-03,
       2.00000000e+01, 2.00000000e+01, 2.00000000e+01])

## noise_gate

In [32]:
# Performs noise-gating of images


# INPUT - data cube

# OUTPUT - noise-gated data cube

# OPTIONAL INPUTS - gate - use gating (default is Weiner Filter)
#                   dct - use Discrete Fourier Transform (much slower)
#                   perc - percentage level of samples to calculate noise level, default 0.5 (median)
#                   gamma - level of SNR to use as a cut-off value, default is 1.5
#                   image_indepen_noise - calculation for removal of image independent additive noise
# TO DO - Parallelise -> WHERE?

import numpy as np

# Recommend using "Utils" Object to hold all relevant functions / image information
class Utils:
    def __init__(self, fourier_image, image):
        self.image = image
        self.fourier_image = fourier_image

    def _estimate_shot_noise(self):
        fac = np.sum(np.sqrt(self.image[self.image>0]))
        betas = np.abs(self.fourier_image)/fac
        return betas

    def _image_indepen_noise(self):
        betas = np.abs(self.fourier_image)
        return betas

    def gate(self, betas, gamm=1.5):
        filt = (1/gamm) * np.abs(self.fourier_image) / betas
        filt = filt[filt>1]
        return filt

    def wiener_filt(self, betas, gamm=1.5):
        filt = np.abs(self.fourier_image) / (gamm*betas+np.abs(self.fourier_image))
        return filt


def noise_gate(data, gated_data=False):
    """
    Noise-gate a data cube.

    :param data: Data cube
    :param gated_data
    """
    sz = data.shape
    nx, ny, nz = sz[0], sz[1], sz[2]

    win_size = 12
    half_win = win_size / 2

    # Apodisation cube for sub-images
    apod = do_apod3d(win_size, win_size, win_size, squared=True)

    noise_elem = nx*ny / win_size / win_size
    noise_arr = 


if __name__ == '__main__':
    data_cube = np.random.rand(100,100,100)

    noise_gate(data_cube)

[ 0.01703709  0.14644661  0.37059048  0.62940952  0.85355339  0.98296291
  0.98296291  0.85355339  0.62940952  0.37059048  0.14644661  0.01703709
 12.         12.         12.        ]
